In [1]:
import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(parent_dir)

from training.training_process import training_process
from models.price_evaluator_xgboost_clasifier import (
    PriceClassifierXGBoostModelHyperparams,
    PriceClassifierXGBoostModel,
)

In [3]:
training_process(
    model_name="xgboost_classifier",
    model_class=PriceClassifierXGBoostModel,
    hyperparameters_class=PriceClassifierXGBoostModelHyperparams,
    intervals_function=lambda x: 10 ** (0.4 * x) + 500 * (x + 1) ** 1.2 + 500 * x**0.8,
    param_grid={
        "learning_rate": [0.14034069550007902],
        "max_depth": [11],
        "n_estimators": [350],
        "min_child_weight": [6],
        "gamma": [1.3736089649503211],
        "subsample": [0.7806217129238506],
        "colsample_bytree": [0.691463437376895],
        "reg_alpha": [0.9541011649041133],
        "reg_lambda": [3.708182521982664],
        "max_delta_step": [5.0],
        "colsample_bynode": [0.7],
        "colsample_bylevel": [0.7],
    },
    gpu_mode=False,
)

585 - 1086
1086 - 2737
2737 - 5482
5482 - 9340
9340 - 14344
14344 - 20548
20548 - 28060
28060 - 37125
37125 - 48332
48332 - 63137
63137 - 85176
85176 - 123562
123562 - 201164
201164 - 375411
375411 - 717877
Evaluated 0.00% of the params.
PriceClassifierXGBoostModelHyperparams(learning_rate=0.14034069550007902, reg_alpha=0.9541011649041133, reg_lambda=3.708182521982664, max_depth=11, n_estimators=350, min_child_weight=6, gamma=1.3736089649503211, subsample=0.7806217129238506, colsample_bytree=0.691463437376895, max_delta_step=5.0, colsample_bynode=0.7, colsample_bylevel=0.7)
ClassificationEvaluationResults(accuracy=0.6608058608058608, precision=0.6597701543546527, recall=0.6608058608058608, f1=0.6599886485962594)


None
